vishesh and shivam 

defeat.

In [ ]:
import torch
import torch.nn as nn

class AudioAttention(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(AudioAttention, self).__init__()
        self.attention = nn.Linear(hidden_dim, 1, bias=False)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, encoder_outputs):
        # Compute attention scores
        attention_scores = self.attention(encoder_outputs)
        attention_weights = self.softmax(attention_scores)
        
        # Apply attention weights
        context_vector = torch.sum(attention_weights * encoder_outputs, dim=1)
        return context_vector, attention_weights

class AudioClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(AudioClassifier, self).__init__()
        self.cnn = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.rnn = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = AudioAttention(input_dim, hidden_dim * 2)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)

    def forward(self, x):
        # Feature extraction with CNN
        x = self.cnn(x)
        x = torch.relu(x)
        x = x.permute(0, 2, 1)  # Prepare for RNN (batch, seq_len, feature_dim)

        # Sequence modeling with RNN
        rnn_outputs, _ = self.rnn(x)

        # Apply attention
        context_vector, attention_weights = self.attention(rnn_outputs)

        # Classification
        logits = self.fc(context_vector)
        return logits, attention_weights